In [5]:
import json
import os
from neo4j import GraphDatabase
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# Neo4j 연결 설정 (환경 변수에서 읽어오기)
uri = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")

# Neo4j 데이터베이스에 연결
driver = GraphDatabase.driver(uri, auth=(username, password))
session = driver.session()

# JSON 파일 경로들
json_files = [
    "D:/programming/python/chunjae/finalproject/kst_pipeline/Preprocessing/01_num_cal.json",
    "D:/programming/python/chunjae/finalproject/kst_pipeline/Preprocessing/02_change_of_relationship.json",
    "D:/programming/python/chunjae/finalproject/kst_pipeline/Preprocessing/03_shape_meas.json",
    "D:/programming/python/chunjae/finalproject/kst_pipeline/Preprocessing/04_data_and_possibility.json",
]


# 대분류 노드 생성 함수
def create_category_node(name):
    query = """
    MERGE (c:Category {name: $name, type: 'Category'})
    RETURN c
    """
    session.run(query, name=name)


# 하위 항목 노드 생성 함수
def create_node(name, parent_name=None, node_type="Item"):
    query = f"""
    MERGE (n:{node_type} {{name: $name, type: $node_type}})
    """
    if parent_name:
        query += """
        WITH n
        MATCH (p {name: $parent_name})
        MERGE (p)-[:HAS_CHILD]->(n)
        """
    session.run(query, name=name, parent_name=parent_name, node_type=node_type)


# JSON 데이터를 처리하고 저장하는 함수
def process_json_data(data, parent_name=None, node_type="Category"):
    for key, value in data.items():
        if isinstance(value, dict):
            item_name = value["name"]
            # 노드 생성
            create_node(item_name, parent_name=parent_name, node_type=node_type)

            # 자식 항목이 있을 경우 재귀적으로 처리
            if "children" in value:
                # 자식 노드에 적합한 타입 지정
                child_node_type = (
                    "Subcategory"
                    if node_type == "Category"
                    else "Leaf" if not value.get("children") else "Item"
                )
                process_json_data(
                    value["children"], parent_name=item_name, node_type=child_node_type
                )
        else:
            # Handle case where value is not a dict but a direct string
            item_name = value
            create_node(item_name, parent_name=parent_name, node_type="Leaf")


# JSON 파일을 처리하고 데이터를 Neo4j에 삽입하는 함수
def insert_json_data(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)
        for key, value in data.items():
            category_name = value["name"]
            # 대분류 노드 생성
            create_category_node(category_name)
            # 자식 항목 처리
            if "children" in value:
                process_json_data(
                    value["children"],
                    parent_name=category_name,
                    node_type="Subcategory",
                )


# 각 JSON 파일을 순차적으로 처리
for file_path in json_files:
    insert_json_data(file_path)

# Neo4j 세션 종료
session.close()